# Day 2: Building an Automation Agent with Claude Agents SDK + MCP

## 🎯 Learning Objectives

By the end of this session, you will:
- Build an agent that creates automation scripts from natural language
- Integrate multiple MCP servers (database + filesystem)
- Test generated scripts automatically
- Handle errors and validate outputs
- Create a production-ready automation workflow

## 💡 What We're Building

An **Automation Agent** that:
1. Reads automation requirements from a SQLite database
2. Generates Python automation scripts based on requirements
3. Saves scripts to the filesystem
4. Tests scripts automatically
5. Updates the database with test results
6. Handles errors and provides feedback

## 🏗️ Architecture Overview

```
┌─────────────────────────────────────────────────┐
│          Automation Agent (Claude)              │
│                                                 │
│  ┌──────────────┐        ┌─────────────────┐   │
│  │   Database   │        │   Filesystem    │   │
│  │  MCP Server  │◄──────►│   MCP Server    │   │
│  └──────────────┘        └─────────────────┘   │
│         ▲                        ▲              │
│         │                        │              │
│         ▼                        ▼              │
│  ┌──────────────────────────────────────────┐  │
│  │        Automation Workflow                │  │
│  │  1. Read Requirements from DB             │  │
│  │  2. Generate Automation Script            │  │
│  │  3. Save Script to Filesystem             │  │
│  │  4. Test Script Execution                 │  │
│  │  5. Update DB with Results                │  │
│  └──────────────────────────────────────────┘  │
└─────────────────────────────────────────────────┘
```


## 🚀 Setup

### Install Dependencies

In [ ]:
!pip install -q claude-agent-sdk anthropic python-dotenv

### Import Required Modules

In [ ]:
import os
import sqlite3
import tempfile
import asyncio
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv
from claude_agent_sdk import (
    tool,
    create_sdk_mcp_server,
    ClaudeSDKClient,
    ClaudeAgentOptions,
    HookMatcher
)

# Load environment variables
load_dotenv()

# Verify API key
if not os.getenv("ANTHROPIC_API_KEY"):
    print("⚠️  Warning: ANTHROPIC_API_KEY not set!")
else:
    print("✅ API key found!")

## 🗄️ Step 1: Create Database with Automation Requirements

First, let's create a SQLite database to store automation requirements:

In [ ]:
def create_automation_database(db_path: str):
    """Create a SQLite database with automation requirements"""
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Create tables
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS automation_tasks (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            task_name TEXT NOT NULL,
            description TEXT NOT NULL,
            requirements TEXT NOT NULL,
            status TEXT DEFAULT 'pending',
            script_path TEXT,
            test_result TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    
    # Insert sample automation tasks
    tasks = [
        (
            "File Organizer",
            "Organize files in a directory by extension",
            """Create a Python script that:
            1. Takes a directory path as input
            2. Scans all files in the directory
            3. Creates subdirectories for each file extension (e.g., 'txt', 'pdf', 'jpg')
            4. Moves files into their respective extension directories
            5. Prints a summary of files moved
            """
        ),
        (
            "Log Analyzer",
            "Analyze log files and extract error patterns",
            """Create a Python script that:
            1. Reads a log file line by line
            2. Extracts all lines containing 'ERROR' or 'CRITICAL'
            3. Groups errors by error type/pattern
            4. Counts occurrences of each error type
            5. Outputs a summary report with error counts
            """
        ),
        (
            "Data Validator",
            "Validate CSV data against rules",
            """Create a Python script that:
            1. Reads a CSV file
            2. Validates that required columns exist
            3. Checks for null/empty values
            4. Validates data types (e.g., numbers, dates)
            5. Generates a validation report with any issues found
            """
        ),
        (
            "Backup Creator",
            "Create timestamped backups of directories",
            """Create a Python script that:
            1. Takes a source directory as input
            2. Creates a ZIP archive with timestamp in filename
            3. Saves backup to a specified backup directory
            4. Removes backups older than 30 days
            5. Prints backup creation status and cleanup summary
            """
        )
    ]
    
    cursor.executemany(
        "INSERT INTO automation_tasks (task_name, description, requirements) VALUES (?, ?, ?)",
        tasks
    )
    
    conn.commit()
    conn.close()
    
    print(f"✅ Created database: {db_path}")
    print(f"📝 Inserted {len(tasks)} automation tasks")

# Create temporary database
temp_dir = tempfile.mkdtemp(prefix="claude_automation_")
db_path = os.path.join(temp_dir, "automation_tasks.db")
scripts_dir = os.path.join(temp_dir, "generated_scripts")
os.makedirs(scripts_dir, exist_ok=True)

create_automation_database(db_path)

print(f"\n📁 Working directory: {temp_dir}")
print(f"📁 Scripts directory: {scripts_dir}")

## 🛠️ Step 2: Create Custom MCP Tools for Database Access

We'll create custom in-process MCP tools for database operations:

In [ ]:
@tool(
    name="get_pending_tasks",
    description="Get all pending automation tasks from the database",
    input_schema={}
)
async def get_pending_tasks(args):
    """Retrieve all tasks with 'pending' status"""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT id, task_name, description, requirements, status
            FROM automation_tasks
            WHERE status = 'pending'
            ORDER BY id
        """)
        
        tasks = cursor.fetchall()
        conn.close()
        
        if not tasks:
            return {"content": [{"type": "text", "text": "No pending tasks found."}]}
        
        # Format tasks as readable text
        result = "Pending Automation Tasks:\n\n"
        for task_id, name, desc, reqs, status in tasks:
            result += f"Task ID: {task_id}\n"
            result += f"Name: {name}\n"
            result += f"Description: {desc}\n"
            result += f"Requirements:\n{reqs}\n"
            result += f"Status: {status}\n"
            result += "=" * 50 + "\n\n"
        
        return {"content": [{"type": "text", "text": result}]}
        
    except Exception as e:
        return {"content": [{"type": "text", "text": f"Error: {str(e)}"}]}


@tool(
    name="update_task_status",
    description="Update the status of an automation task",
    input_schema={
        "task_id": int,
        "status": str,
        "script_path": str,
        "test_result": str
    }
)
async def update_task_status(args):
    """Update task status and results in database"""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            UPDATE automation_tasks
            SET status = ?,
                script_path = ?,
                test_result = ?,
                updated_at = CURRENT_TIMESTAMP
            WHERE id = ?
        """, (
            args.get("status", "completed"),
            args.get("script_path", ""),
            args.get("test_result", ""),
            args.get("task_id")
        ))
        
        conn.commit()
        conn.close()
        
        return {
            "content": [{
                "type": "text",
                "text": f"Successfully updated task {args.get('task_id')} to status: {args.get('status')}"
            }]
        }
        
    except Exception as e:
        return {"content": [{"type": "text", "text": f"Error: {str(e)}"}]}


@tool(
    name="get_all_tasks_summary",
    description="Get a summary of all tasks and their statuses",
    input_schema={}
)
async def get_all_tasks_summary(args):
    """Get summary of all tasks"""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT id, task_name, status, script_path, test_result
            FROM automation_tasks
            ORDER BY id
        """)
        
        tasks = cursor.fetchall()
        conn.close()
        
        result = "Automation Tasks Summary:\n\n"
        for task_id, name, status, script_path, test_result in tasks:
            result += f"[{task_id}] {name}\n"
            result += f"    Status: {status}\n"
            if script_path:
                result += f"    Script: {script_path}\n"
            if test_result:
                result += f"    Test Result: {test_result}\n"
            result += "\n"
        
        return {"content": [{"type": "text", "text": result}]}
        
    except Exception as e:
        return {"content": [{"type": "text", "text": f"Error: {str(e)}"}]}


# Create MCP server with database tools
database_server = create_sdk_mcp_server(
    name="automation-db",
    version="1.0.0",
    tools=[get_pending_tasks, update_task_status, get_all_tasks_summary]
)

print("✅ Created database MCP server with tools:")
print("   • get_pending_tasks")
print("   • update_task_status")
print("   • get_all_tasks_summary")

## 🤖 Step 3: Create the Automation Agent

Now let's create an agent that combines database tools with filesystem access:

In [ ]:
async def run_automation_agent():
    """Run the automation agent to create and test scripts"""
    
    # Configure agent with both database and filesystem access
    options = ClaudeAgentOptions(
        system_prompt="""You are an expert automation engineer. Your task is to:
        
        1. Read pending automation tasks from the database using get_pending_tasks
        2. For EACH task, create a Python script that fulfills the requirements
        3. Save each script to the filesystem with a descriptive filename
        4. Create a simple test for each script
        5. Update the database with the task status and results
        
        Guidelines for script creation:
        - Write clean, well-documented Python code
        - Include proper error handling
        - Add docstrings and comments
        - Make scripts executable with proper __main__ guards
        - Include usage examples in comments
        
        Always verify your work by testing the scripts you create.
        """,
        
        mcp_servers={
            "database": database_server,  # In-process database tools
            "filesystem": {  # External filesystem server
                "type": "stdio",
                "command": "npx",
                "args": ["-y", "@modelcontextprotocol/server-filesystem", scripts_dir]
            }
        },
        
        max_turns=20  # Allow multiple iterations for complex workflows
    )
    
    print("\n" + "=" * 70)
    print("🤖 AUTOMATION AGENT STARTED")
    print("=" * 70)
    print(f"📁 Scripts will be saved to: {scripts_dir}")
    print(f"🗄️  Database: {db_path}")
    print("=" * 70 + "\n")
    
    async with ClaudeSDKClient(options=options) as client:
        # Start the automation workflow
        await client.query("""
        Please complete the following automation workflow:
        
        1. Get all pending tasks from the database
        2. For the first pending task:
           a. Create a Python script that implements the requirements
           b. Save the script with a descriptive filename
           c. Verify the script was saved correctly
           d. Update the task status to 'completed' with the script path
        
        Take your time and be thorough. Explain what you're doing at each step.
        """)
        
        # Stream the response
        async for msg in client.receive_response():
            print(msg, end="", flush=True)
    
    print("\n\n" + "=" * 70)
    print("✅ AUTOMATION AGENT COMPLETED")
    print("=" * 70)

# Run the agent
await run_automation_agent()

## 📊 Step 4: Verify Results

Let's check what scripts were created and the database status:

In [ ]:
def verify_results():
    """Verify the automation results"""
    
    print("\n" + "=" * 70)
    print("📊 VERIFICATION RESULTS")
    print("=" * 70 + "\n")
    
    # Check generated scripts
    print("📁 Generated Scripts:")
    scripts = list(Path(scripts_dir).glob("*.py"))
    if scripts:
        for script in scripts:
            size = script.stat().st_size
            print(f"   ✅ {script.name} ({size} bytes)")
    else:
        print("   ⚠️  No scripts found")
    
    print("\n" + "=" * 70)
    
    # Check database status
    print("\n🗄️  Database Status:")
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute("""
        SELECT id, task_name, status, script_path
        FROM automation_tasks
        ORDER BY id
    """)
    
    tasks = cursor.fetchall()
    for task_id, name, status, script_path in tasks:
        status_icon = "✅" if status == "completed" else "⏳"
        print(f"   {status_icon} [{task_id}] {name}")
        print(f"       Status: {status}")
        if script_path:
            print(f"       Script: {script_path}")
    
    conn.close()
    
    print("\n" + "=" * 70)

verify_results()

## 🎯 Step 5: Advanced - Process Multiple Tasks

Let's extend the agent to process multiple tasks in one go:

In [ ]:
async def process_all_tasks():
    """Process all pending tasks automatically"""
    
    options = ClaudeAgentOptions(
        system_prompt="""You are an expert automation engineer. Process ALL pending tasks:
        
        For EACH pending task:
        1. Read the task requirements
        2. Create a complete, working Python script
        3. Save the script with format: task_{id}_{name}.py
        4. Update the database with completion status
        
        Work through tasks one at a time, being thorough with each.
        """,
        
        mcp_servers={
            "database": database_server,
            "filesystem": {
                "type": "stdio",
                "command": "npx",
                "args": ["-y", "@modelcontextprotocol/server-filesystem", scripts_dir]
            }
        },
        
        max_turns=50  # Allow many iterations for multiple tasks
    )
    
    print("\n🚀 Processing all pending tasks...\n")
    
    async with ClaudeSDKClient(options=options) as client:
        await client.query("""
        Process ALL pending automation tasks:
        1. Get the list of pending tasks
        2. For each task, create the automation script
        3. Save each script to the filesystem
        4. Update the database status for each completed task
        5. Provide a final summary of all work completed
        """)
        
        async for msg in client.receive_response():
            print(msg, end="", flush=True)
    
    print("\n\n✅ All tasks processed!\n")

# Uncomment to process all tasks
# await process_all_tasks()
# verify_results()

## 🎣 Step 6: Adding Monitoring Hooks

Let's add hooks to monitor the agent's actions:

In [ ]:
# Track agent activity
activity_log = []

async def logging_hook(input_data, tool_use_id, context):
    """Log all tool usage"""
    tool_name = input_data.get("tool_name", "unknown")
    timestamp = datetime.now().strftime("%H:%M:%S")
    
    log_entry = f"[{timestamp}] Tool used: {tool_name}"
    activity_log.append(log_entry)
    print(f"\n🔍 {log_entry}")
    
    return {}  # Allow the operation


async def agent_with_monitoring():
    """Run agent with activity monitoring"""
    
    options = ClaudeAgentOptions(
        system_prompt="""Create automation scripts from database requirements.
        Process one task and update its status.""",
        
        mcp_servers={
            "database": database_server,
            "filesystem": {
                "type": "stdio",
                "command": "npx",
                "args": ["-y", "@modelcontextprotocol/server-filesystem", scripts_dir]
            }
        },
        
        hooks={
            "PreToolUse": [
                HookMatcher(matcher="*", hooks=[logging_hook])
            ]
        },
        
        max_turns=15
    )
    
    print("\n🎣 Starting monitored automation agent...\n")
    
    async with ClaudeSDKClient(options=options) as client:
        await client.query("Get a pending task, create its script, and update the database.")
        
        async for msg in client.receive_response():
            print(msg, end="", flush=True)
    
    print("\n\n📊 Activity Log:")
    for entry in activity_log:
        print(f"   {entry}")

# Run with monitoring
# await agent_with_monitoring()

## 🧹 Cleanup

In [ ]:
import shutil

# Show final results before cleanup
print("\n📁 Final Directory Contents:")
for item in Path(scripts_dir).glob("*"):
    print(f"   {item.name}")

# Cleanup (uncomment to remove temp files)
# shutil.rmtree(temp_dir)
# print(f"\n✅ Cleaned up: {temp_dir}")

## 📚 Key Takeaways

### What We Learned

1. **Multi-Server Integration**
   - Combining in-process MCP (database) with external MCP (filesystem)
   - Coordinating between different tool types

2. **Complex Workflows**
   - Multi-step automation processes
   - Database-driven task management
   - Automatic script generation and testing

3. **Production Patterns**
   - Error handling in tools
   - Status tracking and updates
   - Activity logging with hooks

4. **Agent Design**
   - Clear system prompts for complex tasks
   - Appropriate `max_turns` for workflows
   - Monitoring and control mechanisms

### Real-World Applications

This pattern can be adapted for:
- **DevOps Automation**: Generate deployment scripts from requirements
- **Data Pipeline Creation**: Build ETL scripts from specifications
- **Test Automation**: Generate test cases from user stories
- **Documentation**: Create docs from code analysis
- **Migration Tools**: Build migration scripts from schema analysis

## 🎯 Next Steps

1. **Extend the Database Schema**
   - Add priority levels
   - Track execution history
   - Store error logs

2. **Improve Script Testing**
   - Add unit test generation
   - Implement actual script execution
   - Validate outputs

3. **Add More MCP Servers**
   - Git integration for version control
   - Slack/email for notifications
   - Cloud storage for script deployment

4. **Production Deployment**
   - Add authentication
   - Implement rate limiting
   - Create REST API wrapper
   - Add monitoring and alerting

## 📖 Resources

- [Claude Agents SDK](https://docs.claude.com/en/api/agent-sdk/overview)
- [MCP Servers](https://github.com/modelcontextprotocol/servers)
- [SQLite MCP Server](https://github.com/modelcontextprotocol/servers/tree/main/src/sqlite)
- [Production Best Practices](https://docs.anthropic.com/claude/docs/production-guide)
